In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

## Loading the dataset and define the test set relative size

In [2]:
images_path = "/kaggle/input/kittiroadsegmentation/training/image_2"
masks_path = "/kaggle/input/kittiroadsegmentation/training/gt_image_2"

test_set_fraction = 0.20 # 20% of images in the test set
train_set_fraction = 0.80 # 80% of images in the train set

In [23]:

import glob
train_images_filenames = (glob.glob(f"{images_path}/*.png"))[:250]
print(len(train_images_filenames))

250


## Set random seed

In [3]:
np.random.seed(0)

## Load the images and masks

In [4]:
train_set_size = int(len(os.listdir(images_path)) * train_set_fraction)
test_set_size = int(len(os.listdir(masks_path)) * test_set_fraction)

print(f"Number of images in the train set = {train_set_size} and number of images in the test set = {test_set_size}")

Number of images in the train set = 231 and number of images in the test set = 76


In [5]:
def load_images(path, mask = False, verbose = False):
    result = []
    images = os.listdir(path)
    
    if mask is True:
        images = [entry for entry in images if "road" in entry]
    
    images = sorted(images)
    if verbose is True:
        print(f"Processing {len(images)} images")
    
    
    for img in images:
        im = Image.open(os.path.join(path, img))
        resized_im = im.resize((128, 128))
        res = np.asarray(resized_im)
        result.append(res)
    
    return np.array(result)

In [6]:
data = load_images(images_path, mask = False, verbose = True)

Processing 289 images


In [7]:
true_mask = load_images(masks_path, mask = True, verbose = True)

Processing 289 images


## Convert masks to binary format (Road and no road)

In [8]:
road_label = np.array([255, 0, 255])

In [9]:
def convert_to_binary_mask(true_mask):
    num_masks = len(true_mask)
    for i in range(num_masks):
        cond = np.all(true_mask[i] == road_label, axis = 2)
        true_mask[i] = true_mask[i] * cond[..., np.newaxis]

In [10]:
convert_to_binary_mask(true_mask)

## Preprocessing of images and masks

In [11]:
true_mask=np.dot(true_mask[..., :3], [0.2989, 0.5870, 0.1140])
true_mask=np.expand_dims(true_mask, axis=-1)

In [12]:
true_mask[true_mask != 0.0]=1.0

In [13]:
data  = data / 255.0

In [14]:
print(f"Shape of data = {data.shape} and shape of masks = {true_mask.shape}")

Shape of data = (289, 128, 128, 3) and shape of masks = (289, 128, 128, 1)


In [15]:
data=data.astype(np.float32)
true_mask=true_mask.astype(np.float32)

## Saving the numpy arrays as `.npy` files

In [16]:
np.save('images.npy', data)
np.save('binary_masks.npy', true_mask)

## Load BLIP Model

In [17]:
from PIL import Image

def load_image(file_name):
    im = Image.open(file_name).convert("RGB")
    return im

In [18]:
!pip install salesforce-lavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 74.1 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for einops>=0.4.1 from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 22.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for ftfy from https://files.pythonhosted.org/packages/91/f8/dfa32d06cfcbdb76bc46e0f5d69c537de33f4cedb1a15cd4746ab45a6a26/ftfy-6.1.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [19]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from lavis.models import load_model_and_preprocess

model, vis_processors, txt_processors = load_model_and_preprocess(name="blip_feature_extractor", model_type="base", is_eval=True, device=device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


100%|██████████| 1.97G/1.97G [00:08<00:00, 242MB/s]


In [20]:
def get_blip_embeddings(raw_image):
    global model, vis_processors, device
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    sample = {"image": image, "text_input": None}
    features_image = model.extract_features(sample, mode="image")
    return features_image.image_embeds

In [24]:
from tqdm import tqdm

all_blip_embeddings = []

for file_name in tqdm(train_images_filenames):
    raw_image = load_image(file_name)
    embeds = get_blip_embeddings(raw_image)
    all_blip_embeddings.append(embeds)

100%|██████████| 250/250 [00:11<00:00, 22.31it/s]


In [25]:
sum_similarities = 0.0

N = len(all_blip_embeddings)
# all_blip_embeddings = np.array(all_blip_embeddings)

for i in tqdm(range(0, N - 1)):
    for j in range(i+1, N):
        u_x = all_blip_embeddings[i]
        u_y = all_blip_embeddings[j]
        sim = u_x[:,0,:] @ u_y[:,0,:].t()
        sum_similarities += float(sim[0][0])

100%|██████████| 249/249 [00:01<00:00, 142.07it/s]


In [27]:
sum_similarities

7493446.417144775

In [28]:
den = (N*(N-1))/2

In [30]:
avg = sum_similarities/den

In [31]:
avg

240.75329854280403